In [34]:
from min_acfg import Structure, calculate_all_utilities, find_core_stable_coalition


In [35]:
n = 5
N = range(n)
F = {
    0: [1,2],
    1: [0,2],
    2: [0,1,3,4],
    3: [2,4],
    4: [2,3]
}


In [36]:
find_core_stable_coalition(N,F)




[[0, 1, 2, 3, 4]]